In [4]:
import numpy as np
import pandas as pd
import math


def calEntropy(inDataFrame):
    #约定，通常把最后一列作为标签
    labelSeries=inDataFrame.iloc[:,-1]
    freqSum= labelSeries.value_counts()
    #记录的总条数
    sumRecord=inDataFrame.shape[0]
    entropy=-(freqSum/sumRecord)*((freqSum/sumRecord).apply(math.log2))
    sumentropy=entropy.sum()
    return sumentropy

def createDataSet():
    inputDic={'no surfacing':[1,1,1,0,0],
    'flippers':[1,1,0,1,1],
    'fish':['yes','yes','yes','no','no']}
    dataSet= pd.DataFrame(inputDic)
    return dataSet

def bestSplit(dataset):
    bestColIndex=-1
    #计算原始数据集合的信息熵
    setEntropy=calEntropy(dataset)
    #循环各个列，分别计算各个列的修正信息熵、信息增益
    colnum=dataset.shape[1]
    sumrecord=dataset.shape[0]
    maxgain=0
    for colindex in range(0,colnum):
        #计算修正信息熵
        colset=dataset.iloc[:,colindex]
        colValueFreq=colset.value_counts()
        colEntropy=0
        for colvalueindex in colValueFreq.index:
            colValueSum=colValueFreq[colvalueindex]
            #取出等于该属性值的数据
            boolIndex=colset==colvalueindex
            valueSet=dataset[boolIndex]
            valueEntropy=calEntropy(valueSet)
            correctEntropy=(colValueSum/sumrecord)*valueEntropy
            colEntropy+=correctEntropy
        #计算列的信息增益
        colgain=setEntropy-colEntropy
        #比较信息增益，挑选信息增益最大的特征，返回特征列索引
        if(colgain>maxgain):
            maxgain=colgain
            bestColIndex=colindex
    return bestColIndex
def splitDataSet(dataset,colindex,colvalue):
    filter=dataset.iloc[:,colindex]==colvalue
    selected= dataset.loc[filter,:]
    return selected

"""
递归分类
"""
def classify(dataset):
    #如果没有特征可供选择，或者所有的数据都是同一类别，可以不用划分
    featurelist=list(dataset.columns)
    classlist=dataset.iloc[:,-1].value_counts()
    if classlist[0]==dataset.shape[0] or dataset.shape[1]==1:
        return classlist.index[0]
    #选择列标签
    colindex=bestSplit(dataset)
    collabel=featurelist[colindex]
    #决策树
    mytree={collabel:{}}
    del featurelist[colindex]
    valuelist=set(dataset.iloc[:,colindex])
    for value in valuelist:
        mytree[collabel][value]=classify(splitDataSet(dataset,colindex,value))
    return mytree

- 决策树的存储
 使用 numpy 的npy文件进行存储，然后再调用load函数进行加载

In [14]:
 
"""
 分类单条记录的类别
"""
def classifyrecord(mytree,labels,record):
    firstkey= next(iter(mytree))
    valuedic= mytree[firstkey]
    #判断特征
    featureindex=labels.index(firstkey)
    featurevalue=record[featureindex]
    for key in valuedic.keys():
        if featurevalue==key:
            if type(valuedic[key])==dict:
                classlabel=classifyrecord(valuedic[key],labels,record)
            else:
                classlabel=valuedic[key]
    return classlabel


    
def acc_classify(train,test):
    #创建决策树
    decisiontree=classify(train)
    columnlabels=list(test.columns)
    #分类数据
    classifylabels=[]
    for rowindex in range(0,test.shape[0]):
        record=test.iloc[rowindex,:-1]
        label= classifyrecord(decisiontree,columnlabels,record)
        classifylabels.append(label)
    #计算准确率
    test['predict']=classifylabels
    accuracy=(test.iloc[:,-1]==test.iloc[:,-2]).mean()
    return accuracy

In [15]:
dataset=createDataSet()
train=dataset
test=dataset.iloc[:3,:]
acc_classify(train,test)

1.0